In [1]:
import os
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv

load_dotenv()

os.environ["OPENAI_API_KEY"] = os.getenv('OPENAI_API_KEY')
chat = ChatOpenAI(
    openai_api_key=os.environ["OPENAI_API_KEY"],
    model='gpt-3.5-turbo'
)

/Users/zaidsaheb/Desktop/projects/xppcoder/venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
from langchain.schema import (
    SystemMessage,
    HumanMessage,
    AIMessage
)

messages = [
    SystemMessage(content="You are a helpful assistant."),
    HumanMessage(content="Hi AI, how are you today?"),
    AIMessage(content="I'm great thank you. How can I help you?"),
    HumanMessage(content="I'd like to understand what love is.")
]

In [4]:
res = chat(messages)
print(res.content)

/Users/zaidsaheb/Desktop/projects/xppcoder/venv/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


Love is a complex and multifaceted emotion that can be experienced in many different ways. It involves strong feelings of affection, care, and connection towards someone or something. Love can be romantic, platonic, familial, or even towards pets or activities. It often involves a deep sense of empathy, understanding, and support for the other person or thing. Love can bring joy, fulfillment, and a sense of purpose to our lives, and it is often considered one of the most powerful and positive emotions we can experience.


In [ ]:
# add latest AI response to messages
messages.append(res)

# now create a new user prompt
prompt = HumanMessage(
    content="Do you believe in love?"
)
# add to messages
messages.append(prompt)

# send to chat-gpt
res = chat(messages)

print(res.content)

In [5]:
# add latest AI response to messages
messages.append(res)

# now create a new user prompt
prompt = HumanMessage(
    content="What does page 69 of Extending Microsoft Dynamics 365 Finance and Supply Chain Management Cookbook Second Edition by Simon Buxton talk about?"
)
# add to messages
messages.append(prompt)

# send to OpenAI
res = chat(messages)

In [23]:
print(res.content)

I don't have access to specific pages or content from the book "Extending Microsoft Dynamics 365 Finance and Supply Chain Management Cookbook Second Edition" by Simon Buxton. However, I can provide a general overview of the topics covered in the book.

The book is a practical guide that focuses on extending and customizing Microsoft Dynamics 365 Finance and Supply Chain Management using various tools and techniques. It covers a wide range of topics related to development, customization, and integration with the Dynamics 365 platform.

Some of the key topics covered in the book may include:

- Customizing forms, tables, and reports in Dynamics 365 Finance and Supply Chain Management
- Extending functionality using X++, C#, and .NET technologies
- Integrating with external systems and services
- Implementing business logic and workflows
- Enhancing security and data management
- Performance optimization and best practices for development

If you are looking for specific information from 

In [6]:
from pinecone import Pinecone

import re
import pdfplumber
import openai
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter


os.environ["PINECONE_API_KEY"] = os.getenv('PINECONE_API_KEY')

pc = Pinecone(api_key=os.environ["PINECONE_API_KEY"])
index = pc.Index("xppcoder")

/Users/zaidsaheb/Desktop/projects/xppcoder/venv/lib/python3.9/site-packages/pinecone/data/index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [7]:
openai.api_key = os.getenv('OPENAI_API_KEY')
MODEL = "text-embedding-ada-002"

In [4]:
import re
from tqdm import tqdm  # Import tqdm for progress tracking

# Define a function to preprocess text
def preprocess_text(text):
    # Replace consecutive spaces, newlines, and tabs
    text = re.sub(r'\s+', ' ', text)
    return text

def process_pdf(file_path):
    # create a loader
    loader = PyPDFLoader(file_path)
    # load your data
    data = loader.load()
    # Split your data up into smaller documents with Chunks
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
    documents = text_splitter.split_documents(data)
    # Convert Document objects into strings with progress tracking
    texts = [str(doc) for doc in documents]
    return texts




In [7]:
def create_embeddings(texts):
    embeddings_list = []
    for text in texts:
        response = openai.embeddings.create(input=text, model=MODEL)
        embedding = response.data[0].embedding  # Access the embedding correctly
        embeddings_list.append(embedding)
    return embeddings_list

# Define a function to upsert embeddings to Pinecone with metadata
def upsert_embeddings_to_pinecone(index, embeddings, ids, texts, batch_size=100):
    for i in range(0, len(embeddings), batch_size):
        batch_embeddings = embeddings[i:i + batch_size]
        batch_ids = ids[i:i + batch_size]
        batch_texts = texts[i:i + batch_size]
        metadata = [{'text': preprocess_text(text)} for text in batch_texts]
        index.upsert(vectors=[(id, embedding, meta) for id, embedding, meta in zip(batch_ids, batch_embeddings, metadata)])




In [6]:
testtexts = process_pdf("ilovepdf_merged.pdf")
print("successfully processed pdf")

successfully processed pdf


In [8]:
# Define a function to create embedding
# Process a PDF and create embeddings
file_paths = ["dynamics365finopsdocs-3501-4000.pdf"]  # Replace with your actual file path
for file_path in file_paths:
  texts = process_pdf(file_path)
  print("processed text for file", file_path)
  embeddings = create_embeddings(texts)
  # Assuming that `file_path` is used as the ID for the entire document; if you want to use unique IDs for each text chunk, modify this accordingly
  ids = [f"{file_path}_chunk_{i}" for i in range(len(embeddings))]
  # Upsert the embeddings to Pinecone with text as metadata
  upsert_embeddings_to_pinecone(index, embeddings, ids, texts)
  print("upserted embeddings for file", file_path)

processed text for file dynamics365finopsdocs-3501-4000.pdf
upserted embeddings for file dynamics365finopsdocs-3501-4000.pdf


In [8]:
from langchain.vectorstores import Pinecone
from langchain.embeddings import OpenAIEmbeddings

text_field = "text"  # the metadata field that contains our text
embed_model = OpenAIEmbeddings(model=MODEL)
# initialize the vector store object
vectorstore = Pinecone(
    index, embed_model.embed_query, text_field
)

/Users/zaidsaheb/Desktop/projects/xppcoder/venv/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.embeddings.openai.OpenAIEmbeddings` was deprecated in langchain-community 0.0.9 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(
/Users/zaidsaheb/Desktop/projects/xppcoder/venv/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.vectorstores.pinecone.Pinecone` was deprecated in langchain-community 0.0.18 and will be removed in 0.2.0. An updated version of the class exists in the langchain-pinecone package and should be used instead. To use it run `pip install -U langchain-pinecone` and import as `from langchain_pinecone import Pinec

In [12]:

query = "How can I add a new field to a form in Dynamics 365 using Chain of Command??"

vectorstore.similarity_search(query, k=5)

[Document(page_content="page_content='point to Add, and then click New it em.\\n2. Click Dynamics 365 It ems > User Int erface > Display menu it em. Set the Name\\nproperty to FMTAvailableV ehicles .\\n3. Click Add.ﾉExpand table\\nﾉExpand table\\nAdd a new menu item that references the new form' metadata={'source': 'dynamics365finopsdocs-5501-6000.pdf', 'page': 101}"),
 Document(page_content='page_content="6. In the Install Dynamics 365 apps dialog box, find the row where the Name field is\\nset to Dual-wr ite Supply Chain s olution . Then follow one of these steps:\\nIf the Status field for the row indicates that the app isn\'t installed, or that an\\nupdate is available, select the row, and then select Next to open an\\ninstallation wizard. Follow the on-screen instructions to install the app.\\nIf the Status field for the row indicates that the app is installed, enabled, and\\nup to date, select Cancel .\\n7. On the Dynamics 365 apps page, find the row where the Name field is set to

In [16]:
def augment_prompt(query: str):
    # get top 3 results from knowledge base
    results = vectorstore.similarity_search(query, k=7)
    # get the text from the results
    source_knowledge = "\n".join([x.page_content for x in results])
    # feed into an augmented prompt
    augmented_prompt = f"""Using the contexts below, answer the query.

    Contexts:
    {source_knowledge}

    Query: {query}"""
    return augmented_prompt

In [17]:
# create a new user prompt
prompt = HumanMessage(
    content=augment_prompt(query)
)
# add to messages
messages.append(prompt)

res = chat(messages)

print(res.content)

To add a new field to a form in Dynamics 365 using Chain of Command, you can follow these steps:

1. In Solution Explorer, create a new form by right-clicking on the project and selecting "Add > New item".
2. Under Installed > Dynamics 365 Items, choose User Interface, then select Form and give it a name. For example, you can name it ISVForm.
3. Add a button to the form by double-clicking the form in the Solution Explorer. In the Design window, right-click and select "New", then choose Button. Set the Text property to ISVButton.

This process allows you to customize the form by adding a new field using the Chain of Command approach in Dynamics 365.


In [18]:
prompt = HumanMessage(
    content="Can you elaborate on using Chain of command and provide me the steps to add a new field to a form in Dynamics 365?"
)

res = chat(messages + [prompt])
print(res.content)

In Dynamics 365, the Chain of Command framework allows you to extend or modify the functionality of existing objects without modifying the code directly. This framework helps you adhere to best practices and maintain upgradability. Here are the general steps to add a new field to a form in Dynamics 365 using Chain of Command:

1. Identify the form: Determine the form to which you want to add a new field. This could be a standard form or a custom form.

2. Create an extension: Create a new extension for the table that corresponds to the form where you want to add the field. This extension will allow you to add the new field without modifying the base table.

3. Add the new field: Within the extension, add the new field definition with the desired properties such as data type, label, and any validation rules.

4. Extend the form: Create an extension for the form itself. In this extension, you can modify the design of the form to include the new field you added in the previous step.

5. B

In [19]:
prompt = HumanMessage(
    content="How can I create an extension of a table in dynamics 365?"
)

res = chat(messages + [prompt])
print(res.content)

To create an extension of a table in Dynamics 365, you can follow these general steps:

1. Identify the base table that you want to extend.
2. Create a new project in Visual Studio.
3. Add a new project item of type "Table extension" or "Data entity extension".
4. Specify the base table that you want to extend in the extension object.
5. Add new fields, methods, or other customizations to the extension object to meet your requirements.
6. Build and deploy the project to your Dynamics 365 environment.
7. Verify that the extension is working as expected by testing it in the Dynamics 365 application.

These steps may vary slightly depending on the specific version of Dynamics 365 and the development tools you are using. Make sure to refer to the official Microsoft documentation for detailed guidance on creating extensions for tables in Dynamics 365.


In [20]:
prompt = HumanMessage(
    content="Where can I click to add a new project item of type table extension?"
)

res = chat(messages + [prompt])
print(res.content)

You can click on the Dynamics 365 Items > User Interface option and then choose to display the menu item. Set the Name property to FMTAvailableVehicles, and then click Add. This will allow you to add a new menu item that references the new form.


In [25]:
prompt = HumanMessage(
    content="Where can I click in Dynamics 365 to get my data as an excel sheet? "
)

res = chat(messages + [prompt])
print(res.content)

You can click on "Open in Microsoft Office" in Dynamics 365 and then select "Export to Excel" to get your data as an Excel sheet.
